In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from commando import ComManDo
from commando.evaluation import generate_figure, test_partial
import matplotlib.pyplot as plt
from mmd_wrapper import mmd_combine
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing

2022-05-19 15:51:54.907079: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-19 15:51:54.907110: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
np.random.seed(42)

In [4]:
dataset_name = 'Patch-Seq'
modality_names = ['Gene Expression', 'EPhys']
human_cells = pd.read_csv("../data/patchseq_human_L23/human_IVSCC_excitatory_L23_consolidated_0131.csv", index_col=0)
human_morph = pd.read_csv("../data/patchseq_human_L23/All_L23_Lockdown_all_raw_features.csv", index_col=0)
ephys = pd.read_csv("../data/patchseq_human_L23/human_mouse_ephys_all_0127.csv", index_col=0)
sample_names = list(set(human_cells.index) & set(human_morph.index) & set(ephys.index))
data1 = human_morph.loc[sample_names]
data2 = ephys.loc[sample_names]
type1 = type2 = np.array(human_cells.loc[sample_names]['contaminationType'])  # medical_conditions, contaminationType

# Labels
labels = [type1, type2]
features = [None, None]

# Preprocessing
data1 = np.nan_to_num(preprocessing.scale(data1, axis=0))
data2 = np.nan_to_num(preprocessing.scale(data2, axis=0))
dataset = [data1, data2]

# Utility
positivize = lambda X: [(1 + preprocessing.scale(1 + x)) / 2 for x in X]

In [5]:
# Reduced Priors
full_priors = np.eye(len(dataset[0]))

random_idx = np.random.choice(range(len(dataset[0])), int(.5 * len(dataset[0])), replace=False)
priors = np.zeros(len(dataset[0]))
priors[random_idx] = 1
half_priors = np.diag(priors)

none_priors = np.zeros((len(dataset[0]), len(dataset[0])))

# Vars
reduced_dim = 32
batch_size = 50
pca_dim = None

# ComManDo

In [6]:
kwargs = {
    'output_dim': reduced_dim,
    'epoch_DNN': 2000,
    'log_DNN': 500,
    'use_early_stop': False,
    'batch_size': batch_size,
    'pca_dim': pca_dim,
    'dist_method': 'euclidean',
}
cm = ComManDo(**kwargs)
cm_data = cm.fit_transform(dataset=dataset)
cm_data_half = ComManDo(**kwargs, P=half_priors, match_result=cm.match_result, PF_Ratio=.5).fit_transform(dataset=dataset)
cm_data_none = ComManDo(**kwargs, P=none_priors, match_result=cm.match_result, PF_Ratio=0).fit_transform(dataset=dataset)

use random seed: 666
Shape of Raw data
Dataset 0: (103, 60)
Dataset 1: (103, 18)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[500/2000] err:0.4074 alpha:0.5585
epoch:[1000/2000] err:0.3769 alpha:0.5754
epoch:[1500/2000] err:0.3589 alpha:0.5865
epoch:[2000/2000] err:0.3369 alpha:0.5959
Finished Matching!
---------------------------------
Train coupled autoencoders
epoch:[500/2000]: loss:1.072408
epoch:[1000/2000]: loss:0.836454
epoch:[1500/2000]: loss:0.802676
epoch:[2000/2000]: loss:0.792664
Finished Mapping!
---------------------------------
ComManDo Done!
Distance: 0.024471538999932818
Correspondence: 0.682020990003366
Mapping: 18.2141852210043
Total: 18.9206777500076

use random seed: 666
Shape of Raw data
Dataset 0: (103, 60)
Dataset 1: (103, 18)
---------------------------------
Train coupled autoencoders
epoch:[500/2000]: loss:1.110876
epoch:[1000/2000]: loss:0.874180
epoch:[1500/2000]: loss:0.792866
epoch:[2000/2000]: l

# Comparison Methods

In [7]:
# Unioncom
uc = ComManDo(project_mode='tsne', output_dim=reduced_dim)
uc_data = uc.fit_transform(dataset=positivize(dataset))

# NLMA
nlma_data, _ = mmd_combine(*dataset, method='maninetcluster', alignment='nonlinear manifold aln', num_dims=reduced_dim)
nlma_data_half, _ = mmd_combine(*dataset, method='maninetcluster', alignment='nonlinear manifold aln', num_dims=reduced_dim, corr=half_priors)

# LMA
lma_data, _, _ = mmd_combine(*dataset, method='maninetcluster', alignment='linear manifold', num_dims=reduced_dim)
lma_data_half, _, _ = mmd_combine(*dataset, method='maninetcluster', alignment='linear manifold', num_dims=reduced_dim, corr=half_priors)

# CCA
cca_data, _, _ = mmd_combine(*dataset, method='maninetcluster', alignment='cca', num_dims=reduced_dim)
cca_data_half, _, _ = mmd_combine(*dataset, method='maninetcluster', alignment='cca', num_dims=reduced_dim, corr=half_priors)

use random seed: 666
Shape of Raw data
Dataset 0: (103, 60)
Dataset 1: (103, 18)
Device: cpu
---------------------------------
Find correspondence between Dataset 1 and Dataset 2
epoch:[500/2000] err:0.4076 alpha:0.5600
epoch:[1000/2000] err:0.3761 alpha:0.5774
epoch:[1500/2000] err:0.3572 alpha:0.5889
epoch:[2000/2000] err:0.3328 alpha:0.5988
Finished Matching!
---------------------------------
Begin finding the embedded space
epoch:[10/100]: loss:1.209706, align_loss:0.090631
epoch:[20/100]: loss:1.112684, align_loss:0.077716
epoch:[30/100]: loss:0.981669, align_loss:0.079693
epoch:[40/100]: loss:1.001930, align_loss:0.071705
epoch:[50/100]: loss:0.990545, align_loss:0.074800
epoch:[60/100]: loss:0.955851, align_loss:0.074231
epoch:[70/100]: loss:0.985599, align_loss:0.074607
epoch:[80/100]: loss:0.935285, align_loss:0.072969
epoch:[90/100]: loss:0.923122, align_loss:0.072070
epoch:[100/100]: loss:0.961299, align_loss:0.066973
Done
---------------------------------
ComManDo Done!
Dis

# Figure Generation

In [8]:
#%%capture --no-display
# https://miro.medium.com/max/800/1*zm6zVTIEvf9uZ0cE4n0Ogg.png
integrated_data = np.array([
    cm_data, cm_data_half,
    nlma_data, nlma_data_half,
    lma_data, lma_data_half,
    cca_data, cca_data_half,
])
integrated_alg_names = np.array([
    'JAMIE [Full Priors]', 'JAMIE [Half Priors]',
    'Nonlinear Manifold\nAlignment [Full]', 'Nonlinear Manifold\nAlignment [Half]',
    'Linear Manifold\nAlignment [Full]', 'Linear Manifold\nAlignment [Half]',
    'CCA [Full]', 'CCA [Half]',
])
integrated_alg_shortnames = np.array([
    'JAMIE [Full]', 'JAMIE [Half]',
    'NLMA [Full]', 'NLMA [Half]',
    'LMA [Full]', 'LMA [Half]',
    'CCA [Full]', 'CCA [Half]',
])
colors = np.array([
    'blue', 'lightblue',
    'red', 'lightcoral',
    'green', 'lightgreen',
    'm', 'violet',
])
alg_groups = np.zeros(len(integrated_data))
alg_groups[1::2] = 1
idx = range(len(integrated_data))
params = {
    'dataset_names': modality_names,
    'feature_names': [f if dim is None else None for dim, f in zip(pca_dim, features)] if pca_dim is not None else 2*[None],
    'integrated_data': integrated_data[idx],
    'integrated_alg_names': integrated_alg_names[idx],
    'integrated_alg_shortnames': integrated_alg_shortnames[idx],
    'colors': colors[idx],
    'alg_groups': alg_groups[idx],
    'raw_data_group': 0,
    'scale': 20,
    # 'dpi': 100,
    'integrated_use_pca': True,
}
fig = generate_figure(cm, dataset, labels, **params)._fig
fig.savefig('./output_figures/' + dataset_name + '.png')

/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:290: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= np.linalg.norm(y)
/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:290: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= np.linalg.norm(y)
/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:290: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= np.linalg.norm(y)
/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:290: RuntimeWarning: invalid value encountered in true_divide
  correlation_coefficient /= np.linalg.norm(y)
/home/thema/miniconda3/lib/python3.9/site-packages/sklearn/feature_selection/_univariate_selection.py:290: RuntimeWarning: invalid value encountered in true